**Table of contents**<a id='toc0_'></a>    
- [Local Path Planning using Dynamic Window Approach (DWA)](#toc1_)    
- [Starting pybullet](#toc2_)    
- [Definition of LiDAR Class](#toc3_)    
- [Definition of Dynamic Window Approach](#toc4_)    
- [Initial Settings](#toc5_)    
- [Setting Parameters for Mobile Robot](#toc6_)    
- [Setting Parameters for LiDAR](#toc7_)    
- [Setting Obstacles](#toc8_)    
- [Setting Parameters for Dynamic Window Approach](#toc9_)    
- [Execution of Dynamic Window Approach](#toc10_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Local Path Planning using Dynamic Window Approach (DWA)](#toc0_)

This notebook explains the dynamic window approach.

(For a manual summarizing the functions available in pybullet, please refer to [here](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf).)

The dynamic window approach is one of the methods for "local path planning," which is a technique for planning a path for a robot to reach a target point while avoiding obstacles.

Typically, local path planning is not used alone but is often combined with "global path planning" methods like A*.

<br>

Specifically,

1. Plan a global path from the initial point to the target point using global path planning.

<img src="../images/MobileRobot/mobile_robot_local_path_planning_dwa/global_path_planning.png" width="50%">  

<br>
<br>

2. Divide the globally planned path into several intermediate points and use local path planning to sequentially travel from "Intermediate Point 1" to "Intermediate Point 2" to "Intermediate Point 3" ... to the "Target Point."

<img src="../images/MobileRobot/mobile_robot_local_path_planning_dwa/local_path_planning.png" width="50%">

In local path planning, the robot travels to the target point while considering the surrounding environment in real-time.  
Therefore, even if obstacles move slightly, the robot can reach the target point while avoiding obstacles without re-planning the global path.


# <a id='toc2_'></a>[Starting pybullet](#toc0_)

In [1]:
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 

startThreads creating 1 threads.
starting thread 0


pybullet build time: Nov 28 2023 23:45:17


started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


# <a id='toc3_'></a>[Definition of LiDAR Class](#toc0_)

In [2]:
import math 
import numpy as np

class Lidar():
    def __init__(self):
        self.replace_item_uniqueIds = None

    # Rotation matrix representing the posture
    def rotation_matrix(self, roll, pitch, yaw):
        R_x = [[1, 0, 0],
                [0, math.cos(roll), -math.sin(roll)],
                [0, math.sin(roll), math.cos(roll)]]
        
        R_y = [[math.cos(pitch), 0, math.sin(pitch)],
                [0, 1, 0],
                [-math.sin(pitch), 0, math.cos(pitch)]]
        
        R_z = [[math.cos(yaw), -math.sin(yaw), 0],
                [math.sin(yaw), math.cos(yaw), 0],
                [0, 0, 1]]
        
        return np.dot(R_z, np.dot(R_y, R_x))

    def CheckHits(self, ray_from_position, ray_orientation, ray_length, angle_resolution_rad, ray_direction_range_rad, offset, draw_debug_line=False):
        """
        Raycast for obstacle detection reflecting posture
        Parameters
        ---
        ray_from_position : list
            List storing the position where the ray starts [x, y, z]
        orientation : list
            Posture of the ray origin [roll, pitch, yaw]
        ray_length : float
            Length of the ray
        angle_resolution_rad : float
            Angle resolution
        ray_direction_range_rad : float
            List storing the range to fire the ray [startRad, endRad]
        offset : float
            Offset from rayFromPosition
        draw_debug_line : bool
            Whether to draw the ray or not
        
        Returns
        ---
        hit_results: tuple
            Results of the raycast
        """
        
        
        R = self.rotation_matrix(ray_orientation[0], ray_orientation[1], ray_orientation[2])
        
        # Number of divisions
        num_divisions = int((ray_direction_range_rad[1] - ray_direction_range_rad[0]) / angle_resolution_rad)

        # Generate ray start/end points for numDivisions within the range of ray_direction_range_rad
        ray_froms = []
        ray_toes = []
        for i in range(num_divisions):
            angle = ray_direction_range_rad[0] + i * angle_resolution_rad
            direction = [math.cos(angle), math.sin(angle), 0]
            
            # Apply rotation matrix to rotate the direction vector
            rotated_direction = np.dot(R, direction)

            # Ray start point
            ray_from = ray_from_position + offset * rotated_direction
            ray_froms.append(ray_from)

            # Ray end point
            ray_to = ray_from + ray_length * rotated_direction
            ray_toes.append(ray_to)
        
        # Perform raycast for numDivisions
        hit_results = pybullet.rayTestBatch(ray_froms, ray_toes)

        # Draw the ray
        if draw_debug_line:
            if self.replace_item_uniqueIds is None:
                self.replace_item_uniqueIds = []
                
                for i in range(num_divisions):
                    isHit = hit_results[i][0]
                    ray_from = ray_froms[i]
                    
                    if isHit != -1:
                        rayHitPosition = hit_results[i][3]
                        self.replace_item_uniqueIds.append(pybullet.addUserDebugLine(ray_from, rayHitPosition, lineColorRGB=[1,0,0]))
                    else:
                        rayToPosition = ray_toes[i]
                        self.replace_item_uniqueIds.append(pybullet.addUserDebugLine(ray_from, rayToPosition, lineColorRGB=[0,1,0]))
            
            else:
                for i in range(num_divisions):
                    isHit = hit_results[i][0]
                    ray_from = ray_froms[i]
                    
                    if isHit != -1:
                        rayHitPosition = hit_results[i][3]
                        pybullet.addUserDebugLine(ray_from, rayHitPosition, lineColorRGB=[1,0,0], replaceItemUniqueId=self.replace_item_uniqueIds[i])
                    else:
                        rayToPosition = ray_toes[i]
                        pybullet.addUserDebugLine(ray_from, rayToPosition, lineColorRGB=[0,1,0], replaceItemUniqueId=self.replace_item_uniqueIds[i])
        return hit_results


ven = Mesa


# <a id='toc4_'></a>[Definition of Dynamic Window Approach](#toc0_)

Dynamic Window Approach (DWA) is a method that evaluates combinations of a robot's linear and angular velocities to select the optimal ones.

The general procedure is as follows:
1. Initialization
   - Initialization is performed in the `__init__` method.
2. Calculation of Vr (Velocity Result)
   - Vr is calculated in the `calc_vr` method.
   - Vr is the intersection of Vs (Velocity Sample), Vd (Velocity dynamic window), and Va (Velocity admissible).
     - Vs: Range of the robot's linear and angular velocities (determined by the robot's physical specifications)
     - Vd: Range of linear and angular velocities the robot can take within a small time interval dt from its current velocities
     - Va: Range of linear and angular velocities at which the robot does not collide with obstacles when operating at its current position
   <img src="../images/MobileRobot/mobile_robot_local_path_planning_dwa/dynamic_window_approach_overview.png" width="50%">
3. Selection of optimal linear and angular velocities
   - The `calc_best_u` method selects the optimal linear and angular velocities from the range of Vr.
     1. Calculation of predicted trajectory
          - The `calc_predicted_trajectory` method calculates the predicted trajectory up to `predict_time` based on the linear and angular velocities.
     2. Calculation of costs: The following costs are calculated for the predicted trajectory.
          - `calc_to_goal_cost` method: Whether the end point of the trajectory is facing the goal direction
          - `calc_speed_cost` method: Whether the trajectory allows the robot to move at high linear and angular velocities
          - `calc_obstacle_cost` method: Whether the trajectory keeps a safe distance from obstacles
     3. The linear and angular velocities that result in the trajectory with the lowest total cost are selected as the control inputs.
4. Steps 2 and 3 are repeated until the goal is reached.
 

In [ ]:
class Vs:
    """
    Definition of Vs (velocity space).
    Defines the range of velocities and angular velocities that the robot can take based on its specifications (the range is basically fixed as it depends on the specifications of the mobile robot).
    """
    def __init__(self, linear_vel_min, linear_vel_max, angle_vel_min, angle_vel_max):
        self.linear_vel_min = linear_vel_min
        self.linear_vel_max = linear_vel_max
        self.angle_vel_min = angle_vel_min
        self.angle_vel_max = angle_vel_max

class Vd:
    """
    Definition of Vd (Velocity dynamic window).
    Defines the range of velocities and angular velocities that the robot can take during a small time interval dt based on its current velocity and angular velocity.
    """
    def __init__(self):
        self.linear_vel_min = 0.0
        self.linear_vel_max = 0.0
        self.angle_vel_min = 0.0
        self.angle_vel_max = 0.0

    def calc_Vd(self, linear_vel_current, angle_vel_current, linear_acc_min, linear_acc_max, angle_acc_min, angle_acc_max, dt):
        self.linear_vel_min = linear_vel_current + linear_acc_min * dt
        self.linear_vel_max = linear_vel_current + linear_acc_max * dt
        self.angle_vel_min = angle_vel_current + angle_acc_min * dt
        self.angle_vel_max = angle_vel_current + angle_acc_max * dt

class Vr:
    """
    Definition of Vr (Velocity result).
    Calculates the intersection of Vs and Vd (the overlapping range of Vs and Vd).
    In the paper, Vr is defined as the intersection of Vs, Vd, and Va, but calculating the entire region of Va is computationally expensive. Therefore, here it is defined as the intersection of Vs and Vd.
    Va is determined only on candidate trajectories in "calc_obstacle_cost" (since it is sufficient to know whether the candidate trajectory is in the Va region, it is not necessary to calculate the entire Va in advance).
    ※ Va (velocity admissible): The range of velocities and angular velocities where the robot does not collide with obstacles when moving at a certain velocity and angular velocity from the current position.
       Unlike Vs and Vd, collision detection is required to determine the region, making it computationally expensive.
    """
    def __init__(self):
        self.linear_vel_min = 0.0
        self.linear_vel_max = 0.0
        self.angle_vel_min = 0.0
        self.angle_vel_max = 0.0

    def calc_Vr(self, vs: Vs, vd: Vd):
        self.linear_vel_min = max(vs.linear_vel_min, vd.linear_vel_min)
        self.linear_vel_max = min(vs.linear_vel_max, vd.linear_vel_max)
        self.angle_vel_min = max(vs.angle_vel_min, vd.angle_vel_min)
        self.angle_vel_max = min(vs.angle_vel_max, vd.angle_vel_max)

class DynamicWindowApproach:

    def __init__(self, x_init,
                       linear_vel_min, linear_vel_max, linear_acc_min, linear_acc_max,
                       angle_vel_min, angular_vel_max, angle_acc_min, angle_acc_max,
                       predict_time, dt,
                       linear_vel_resolution, angle_vel_resolution,
                       to_goal_cost_gain, speed_cost_gain, obstacle_cost_gain,
                       lidar_angle_min, lidar_angle_resolution,
                       robot_radius):
        self.x = x_init
        self.predict_time = predict_time
        self.dt = dt
        self.linear_vel_resolution = linear_vel_resolution
        self.angle_vel_resolution = angle_vel_resolution
        self.to_goal_cost_gain = to_goal_cost_gain
        self.speed_cost_gain = speed_cost_gain
        self.obstacle_cost_gain = obstacle_cost_gain
        self.lidar_angle_min = lidar_angle_min
        self.lidar_angle_resolution = lidar_angle_resolution
        self.robot_radius = robot_radius

        # Velocity space (the range of velocities and angular velocities that the robot can take)
        self.vs = Vs(linear_vel_min, linear_vel_max, angle_vel_min, angular_vel_max)

        self.linear_acc_min = linear_acc_min
        self.linear_acc_max = linear_acc_max
        self.angle_acc_min = angle_acc_min
        self.angle_acc_max = angle_acc_max

    def motion_model(self, x, u, dt):
        """
        Calculates the next state x_next from the current state x and input u using the motion model of a two-wheeled robot on a 2D plane.
        
        Parameters
        ----------
        x : numpy.ndarray
            Current state [x, y, theta]
        u : numpy.ndarray
            Input [v, w]
        dt : float
            Time step

        Returns
        -------
        x_next : numpy.ndarray
            Next state [x, y, theta]
        """
        theta = x[2]
        F = np.array([[1.0, 0.0, 0.0, 0.0, 0.0],
                      [0.0, 1.0, 0.0, 0.0, 0.0],
                      [0.0, 0.0, 1.0, 0.0, 0.0],
                      [0.0, 0.0, 0.0, 0.0, 0.0],
                      [0.0, 0.0, 0.0, 0.0, 0.0]])
        B = np.array([[dt * np.cos(theta), 0.0],
                      [dt * np.sin(theta), 0.0],
                      [0.0, dt],
                      [1.0, 0.0],
                      [0.0, 1.0]])
        x_next = F @ x + B @ u # Motion model without considering friction
        return x_next


    def calc_best_u(self, x, u, lidar_results, goal):
        """
        Calculates the optimal velocity and angular velocity

        Parameters
        ----------
        x : numpy.ndarray
            Current state [x, y, theta]
        u : numpy.ndarray
            Current velocity and angular velocity [v, w]
        lidar_results : list
            Lidar results
        goal : numpy.ndarray
            Goal position [x, y]

        Returns
        -------
        u_best : numpy.ndarray
            Optimal velocity and angular velocity calculated by the dynamic window approach [v, w]
        """
        # Calculate Vr
        vd = Vd()
        vd.calc_Vd(u[0], u[1], self.linear_acc_min, self.linear_acc_max, self.angle_acc_min, self.angle_acc_max, self.dt)
        vr = Vr()
        vr.calc_Vr(self.vs, vd)

        # Search for the optimal velocity and angular velocity
        u_best = np.array([0.0, 0.0])
        min_cost = 1000000
        for linear_vel_candidate in np.arange(vr.linear_vel_min, vr.linear_vel_max, self.linear_vel_resolution):
            for angle_vel_candidate in np.arange(vr.angle_vel_min, vr.angle_vel_max, self.angle_vel_resolution):
                # Calculate the predicted trajectory
                u_candidate = np.array([linear_vel_candidate, angle_vel_candidate])
                trajectory = self.calc_predict_trajectory(x, u_candidate, self.predict_time, self.dt)

                # Calculate the cost
                to_goal_cost = self.calc_to_goal_cost(trajectory, goal)
                speed_cost = self.calc_speed_cost(trajectory, self.vs)
                obstacle_cost = self.calc_obstacle_cost(trajectory, lidar_results, self.robot_radius)
                total_cost = self.to_goal_cost_gain * to_goal_cost + self.speed_cost_gain * speed_cost + self.obstacle_cost_gain * obstacle_cost
                
                # Update the minimum cost
                if total_cost < min_cost:
                    min_cost = total_cost
                    u_best = np.array([linear_vel_candidate, angle_vel_candidate])
        return u_best

    def calc_predict_trajectory(self, x, u, predict_time, dt):
        """
        Calculates the trajectory up to predict_time from the current state x and control input u

        Parameters
        ----------
        x : numpy.ndarray
            Current state [x, y, theta]
        u : numpy.ndarray
            Control input [v, w]
        dt : float
            Time step

        Returns
        -------
        trajectory : numpy.ndarray
            Trajectory [x, y, theta, v, w]
        """
        trajectory = np.array([x])
        current_time = 0.0
        while current_time <= predict_time:
            x = self.motion_model(x, u, dt)
            trajectory = np.vstack((trajectory, x))
            current_time += dt
        return trajectory            
    
    def calc_to_goal_cost(self, trajectory, goal):
        """
        Calculates the cost to the goal
        The cost is the difference in angle between the final position of the trajectory and the goal position (the more the trajectory is directed towards the goal, the lower the cost).

        Parameters
        ----------
        trajectory : numpy.ndarray
            Trajectory [x, y, theta, v, w]
        goal : numpy.ndarray
            Goal position [x, y]

        Returns
        -------
        cost : float
            Cost to the goal
        """
        dx = goal[0] - trajectory[-1, 0]
        dy = goal[1] - trajectory[-1, 1]
        error_angle = math.atan2(dy, dx)
        cost_angle = error_angle - trajectory[-1, 2]
        cost = abs(math.atan2(math.sin(cost_angle), math.cos(cost_angle)))
        return cost
    
    def calc_speed_cost(self, trajectory, vs: Vs):
        """
        Calculates the cost related to speed
        The cost is the difference from the current speed (the higher the speed, the lower the cost).

        Parameters
        ----------
        trajectory : numpy.ndarray
            Trajectory [x, y, theta, v, w]
        vs : Vs
            Velocity space

        Returns
        -------
        cost : float
            Cost related to speed
        """
        cost = vs.linear_vel_max - trajectory[-1, 3]    
        return cost

    def calc_obstacle_cost(self, trajectory, lidar_results, robot_radius):
        """
        Calculates the cost related to obstacles
        The cost is the inverse of the distance to obstacles at each point on the trajectory (the farther from obstacles, the lower the cost).

        Parameters
        ----------
        trajectory : numpy.ndarray
            Trajectory [x, y, theta, v, w]
        lidar_results : list

        Returns
        -------
        cost : float
            Cost related to obstacles
        """
        min_dist = 100000
        for trajectory_point_x, trajectory_point_y , _, _, _ in trajectory:
            for lidar_result in lidar_results:
                hit_position = lidar_result[3] # Position where the lidar hit (in world coordinates)
                # Skip if the lidar did not hit (if not hit, hit_point=[0, 0, 0])
                if hit_position[0] == 0.0 and hit_position[1] == 0.0:
                    continue
                # The closer the distance between the "point on the predicted trajectory" and the "position where the ray hit", the higher the cost, as it is judged that there is an obstacle near the predicted trajectory.
                dx = trajectory_point_x - hit_position[0]
                dy = trajectory_point_y - hit_position[1]
                dist = np.hypot(dx, dy) - robot_radius
                if dist < min_dist:
                    min_dist = dist
        cost = 1.0 / (min_dist + 1e-6)
        return cost


# <a id='toc5_'></a>[Initial Settings](#toc0_)

Set up the environment and generate the robot.

In [4]:
pybullet.resetSimulation() # Reset the simulation space
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # Add path to the data required by pybullet
pybullet.setGravity(0.0, 0.0, -9.8) # Set gravity as on Earth
time_step = 0.1
pybullet.setTimeStep(time_step) # Set the elapsed time per step

# Set the camera position for GUI mode
camera_distance = 6.0
camera_yaw = 180.0 # deg
camera_pitch = -90.1 # deg
camera_target_position = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

# Load the floor
plane_id = pybullet.loadURDF("plane.urdf")

# Load the robot
car_start_pos = [0, 0, 0.1]  # Set the initial position (x, y, z)
car_start_yaw = math.radians(0)
car_start_orientation = pybullet.getQuaternionFromEuler([0, 0, car_start_yaw]) # Convert the initial orientation (roll, pitch, yaw) to quaternion
car_id = pybullet.loadURDF("../urdf/simple_two_wheel_car.urdf", car_start_pos, car_start_orientation)

# Goal point sphere
goal_visual_id = pybullet.createVisualShape(pybullet.GEOM_SPHERE, radius=0.1, rgbaColor=[0,1,0,1])
goal_id = pybullet.createMultiBody(0, -1, goal_visual_id, [0.0, 0.0, 0.0], useMaximalCoordinates=True)

ven = Mesa


# <a id='toc6_'></a>[Setting Parameters for Mobile Robot](#toc0_)

In [5]:
# Joint indices of the left and right wheels
RIGHT_WHEEL_IDX = 0
LEFT_WHEEL_IDX = 1
WHEEL_THREAD = 0.325 # Distance between wheels (match the distance between wheels in "simple_two_wheel_car.urdf")

# <a id='toc7_'></a>[Setting Parameters for LiDAR](#toc0_)

In [6]:
LIDAR_LINK_IDX = 5 # Index of the link from which the ray is emitted
lidar_ray_origin_offset = 0.1 # Offset from the origin of the lidar [m]
lidar_ray_length = 2.0 # Length of the ray
lidar_angle_min = math.radians(-180) # Minimum angle of the ray
lidar_angle_max = math.radians(180) # Maximum angle of the ray
lidar_angle_resolution = math.radians(5) # Angular resolution of the ray

# <a id='toc8_'></a>[Setting Obstacles](#toc0_)

Generate obstacles.

**The control of the dynamic window approach changes depending on the position of the obstacles, so feel free to change or add obstacles to any position.**


In [7]:
# Add obstacle
mass = 200 # kg
box_size = [0.5, 0.5, 0.3]
position = [2.0, 2.0, 0.3]
orientation = [1, 0, 0, 0]
box_collision_id = pybullet.createCollisionShape(pybullet.GEOM_BOX, halfExtents=box_size, physicsClientId=physics_client)
box_visual_id = pybullet.createVisualShape(pybullet.GEOM_BOX, halfExtents=box_size, physicsClientId=physics_client, rgbaColor=[1,0,0,1]) 
box_body_id = pybullet.createMultiBody(mass, box_collision_id, box_visual_id, position, orientation, physicsClientId=physics_client)

# <a id='toc9_'></a>[Setting Parameters for Dynamic Window Approach](#toc0_)

Set parameters related to the dynamic window approach.

**Try changing each parameter to see how the robot behaves.**

---

Note: Depending on the position of obstacles and the target location, the dynamic window approach may fall into a local optimum and fail to reach the target point.

Therefore, as mentioned at the beginning,
- Find a global path using global path planning
- → Divide the obtained path into several intermediate points
- → "Move towards Intermediate Point 1" → "Once reaching Intermediate Point 1, move towards Intermediate Point 2" → ... Repeat until reaching the target point (dividing into intermediate points reduces the possibility of falling into a local optimum)

Such a procedure is generally used. (In this notebook, global path planning is not performed.)

---


In [8]:
# Parameters related to dynamic window
linear_vel_min = -0.5 # Minimum speed of the robot [m/s]
linear_vel_max = 1.0 # Maximum speed of the robot [m/s]
linear_acc_min = -0.5 # Minimum acceleration of the robot [m/s^2]
linear_acc_max = 0.5 # Maximum acceleration of the robot [m/s^2]
angle_vel_min = math.radians(-80) # Minimum angular velocity of the robot [rad/s]
angle_vel_max = math.radians(80) # Maximum angular velocity of the robot [rad/s]
angle_acc_min = math.radians(-40) # Minimum angular acceleration of the robot [rad/s^2]
angle_acc_max = math.radians(40) # Maximum angular acceleration of the robot [rad/s^2]
predict_time = 2.0 # Prediction time [s] (how far ahead to calculate the predicted trajectory)
linear_vel_resolution = 0.02 # Speed resolution [m/s]
angle_vel_resolution = 0.02 # Angular velocity resolution [rad/s]
robot_radius = 0.1 # Radius of the robot [m]

# Parameters related to cost
to_goal_cost_gain = 0.15 # Weight of the cost to the goal (the more the direction differs from the goal, the higher the cost)
speed_cost_gain = 1.0 # Weight of the cost related to speed (the slower the speed, the higher the cost)
obstacle_cost_gain = 0.7 # Weight of the cost related to obstacles (the closer to obstacles, the higher the cost)

# Setting the goal position
goal = np.array([1, 5, 0])
# goal = np.array([4, 4, 0])
# goal = np.array([3.5, 2, 0])

# Place the spherical object indicating the goal position at the goal position
pybullet.resetBasePositionAndOrientation(goal_id, goal, [0, 0, 0, 1])

# <a id='toc10_'></a>[Execution of Dynamic Window Approach](#toc0_)

Execute the dynamic window approach based on the specified parameters.

In [ ]:
import sys

# Initialize debug drawing
pybullet.removeAllUserDebugItems()

# Set the robot to the initial position
pybullet.resetBasePositionAndOrientation(car_id, car_start_pos, car_start_orientation)

# Initialize lidar
lidar = Lidar()

# Initialize dynamic window approach
dwa = DynamicWindowApproach(x_init=np.array(car_start_pos), 
                            linear_vel_min=linear_vel_min, linear_vel_max=linear_vel_max, linear_acc_min=linear_acc_min, linear_acc_max=linear_acc_max,
                            angle_vel_min=angle_vel_min, angular_vel_max=angle_vel_max, angle_acc_min=angle_acc_min, angle_acc_max=angle_acc_max,
                            predict_time=predict_time, dt=time_step,
                            linear_vel_resolution=linear_vel_resolution, angle_vel_resolution=angle_vel_resolution,
                            to_goal_cost_gain=to_goal_cost_gain, speed_cost_gain=speed_cost_gain, obstacle_cost_gain=obstacle_cost_gain,
                            lidar_angle_min=lidar_angle_min, lidar_angle_resolution=lidar_angle_resolution,
                            robot_radius=robot_radius)


# Initial control input
v_init = 0.0
omega_init = math.radians(0.0)
u = np.array([v_init, omega_init])

# Initial state
x = np.array([car_start_pos[0], car_start_pos[1], car_start_yaw, v_init, omega_init])

while True:
    # Get lidar information
    lidar_position = pybullet.getLinkState(car_id, LIDAR_LINK_IDX) # Get information of lidar link
    ray_from_position = [lidar_position[0][0], lidar_position[0][1], lidar_position[0][2]] # Set the starting point of the ray to the origin of the lidar link
    lidar_orientation = list(pybullet.getEulerFromQuaternion(lidar_position[1])) 
    lidar_results = lidar.CheckHits(ray_from_position, lidar_orientation, lidar_ray_length,  lidar_angle_resolution, [lidar_angle_min, lidar_angle_max], lidar_ray_origin_offset, draw_debug_line=True)
    
    # Main process of dynamic window approach ========================================
    u = dwa.calc_best_u(x, u, lidar_results, goal) # Calculate the optimal control input
    # =========================================================================
    v = u[0]
    omega = u[1]
    sys.stdout.write(f"\r v: {round(v, 3)} omega: {round(omega, 3)}       ")

    # Calculate the command speed of the left and right wheels from the translational speed and rotational speed
    right_wheel_velocity = v + omega * WHEEL_THREAD / 2
    left_wheel_velocity = v - omega * WHEEL_THREAD / 2

    # Set the speed
    pybullet.setJointMotorControl2(car_id, RIGHT_WHEEL_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=right_wheel_velocity)
    pybullet.setJointMotorControl2(car_id, LEFT_WHEEL_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=left_wheel_velocity)

    # Advance the simulation by one step
    pybullet.stepSimulation()

    # Update the state (here, "x, y, yaw" are the true values obtained directly from the simulation. In practice, the values estimated using odometry, etc. are used)
    car_position, car_orientation = pybullet.getBasePositionAndOrientation(car_id)
    car_euler = pybullet.getEulerFromQuaternion(car_orientation)
    car_yaw = car_euler[2]
    x = np.array([car_position[0], car_position[1], car_yaw, v, omega], dtype=np.float64)

    # End if the robot reaches near the goal
    goal_dis = 0.3
    if np.linalg.norm(goal[:2] - x[:2]) < goal_dis:
        pybullet.addUserDebugText("GOAL!!", [goal[0]-0.7, goal[1]-0.8, 0.3], [0, 0, 1], textSize=2)
        break

 v: 0.89 omega: 0.878       